# PROJECT UNDERSTANDING

**Introduction**

Sepsis is a life-threatening medical condition that occurs when the body's response to an infection injures its own tissues and organs. It is a leading cause of death in hospitals worldwide, and its incidence is increasing. Early diagnosis and treatment of sepsis are crucial for improving patient outcomes. However, sepsis can be difficult to diagnose in its early stages, as its symptoms can be subtle and nonspecific.

Early diagnosis and treatment of sepsis is critical for improving patient outcomes. However, sepsis can be difficult to diagnose early because its symptoms are often non-specific and can overlap with other conditions. As a result, sepsis is often underdiagnosed or misdiagnosed, leading to delayed treatment and worse patient outcomes.

A sepsis prediction model could help to improve the early diagnosis and treatment of sepsis by identifying patients who are at high risk of developing the condition. This could be done by using machine learning algorithms to analyze patient data from electronic health records (EHRs).



Read About Data Columns **[here](https://github.com/fantastic-rambo/Embedding-Machine-Learning-Model-in-FastAPI/blob/main/data/README.md)**

**About the Datasets:**

* **ID:**                            Unique patient identifier.
* **PRG** (Plasma glucose):          Measurement of plasma glucose levels.
* **PL** (Blood Work Result-1):      First blood work result (in mu U/ml).
* **PR** (Blood Pressure):           Blood pressure measurement (in mm Hg).
* **SK** (Blood Work Result-2):       Second blood work result (in mm).
* **TS** (Blood Work Result-3):       Third blood work result (in mu U/ml).
* **M11** (Body mass index):          Body mass index calculated as weight in kg divided by the square of height in meters.
* **BD2** (Blood Work Result-4):      Fourth blood work result (in mu U/ml).
* **Age**: Age of the patient in years.

* **Insurance**:              Binary indicator of whether the patient holds a valid insurance card.

* **Sepsis**: Binary outcome indicating the development of sepsis in the ICU (Positive or Negative). otherwise


## Goal Of Project
The goal of the project is to develop a model that can predict whether or not a patient in the ICU will develop sepsis. This model could be used to identify patients who are at high risk of developing sepsis, allowing clinicians to initiate early treatment and improve patient outcomes.





## Hypothesis
**Null Hypothesis (H0)**

Age does not determine whether a patient will develop Sepsis.


**Alternative Hypothesis (Ha)**

Age determines whether a patient will develop Sepsis.




## Analytical Questions

* What is the average age of those developing sepsis?

* what is the Average BMI of those developing sepsis?

* Are there discernible demographic patterns among patients who develop sepsis, such as age, insurance status, or other demographic variables?

* What is the correlation between different medical indicators (e.g., plasma glucose, blood pressure) and the likelihood of developing sepsis?

* Do specific blood work results exhibit a noticeable impact on the probability of sepsis development?

* Is there a significant association between insurance status and the risk of developing sepsis in ICU patients?

* How well can we predict the likelihood of sepsis based on the available variables in the dataset?

* Are there any temporal trends in the occurrence of sepsis or changes in the distribution of key variables over time?

* How do different predictive models perform in estimating the likelihood of sepsis?

# DATA UNDERSTANDING

In [ ]:
# Installations
!pip install -U imbalanced-learn


In [ ]:
import pandas as pd                       # For data manipulation and analysis
import numpy as np                        # For numerical operations
import matplotlib.pyplot as plt           # For data visualization
import seaborn as sns                     # For statistical data visualization
import re
import threadpoolctl
from imblearn.over_sampling import SMOTE

#import warning


#Libraries for feature scaling
from sklearn.preprocessing import StandardScaler

#Libraries for Validation
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import confusion_matrix
#from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics   #Import scikit-learn metrics module for accuracy calculation

#Libraries for Training model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn import tree

import joblib

In [ ]:
#Check numpy and pandas version

print("Numpy version: ", np.__version__)
print("Pandas version: ",pd.__version__)

In [ ]:
#loading data from Github

train_data = pd.read_csv("https://raw.githubusercontent.com/fantastic-rambo/Embedding-Machine-Learning-Model-in-FastAPI/main/data/Paitients_Files_Train.csv")
test_data = pd.read_csv("https://raw.githubusercontent.com/fantastic-rambo/Embedding-Machine-Learning-Model-in-FastAPI/main/data/Paitients_Files_Test.csv")

## **Data Understanding**

In [ ]:
train_data.head()

In [ ]:
test_data.head()

### EDA

In [ ]:
def display_dataset_info(train_data, test_data):
    """
    Display information about the train and test datasets.

    Parameters:
    - train_df: DataFrame, the training dataset.
    - test_df: DataFrame, the testing dataset.
    """
    print("Train Dataset Info:")
    train_data.info()

    print("\nTest Dataset Info:")
    test_data.info()

# Assuming you have train_data and test_data DataFrames
display_dataset_info(train_data, test_data)

In [ ]:
train_data.shape, test_data.shape          # checking for the shapes

From the training dataset, I extracted information revealing a total of 599 patient records, all complete with no missing values. The dataset comprises 11 columns, including the target variable 'Sepsis.'

Also, From the test dataset, it is observed that there are a total of 169 entries. The dataset spans 10 columns, and it is noteworthy that the 'sepsis' target column is not present, indicating that sepsis outcome labels are not included in the test data.

In [ ]:
train_data.duplicated().sum(), test_data.duplicated().sum()   #checking for duplicates

From The Above codes, (0, 0) depicts that both the Train and Test datasets do not possess duplicated rows.

In [ ]:
train_data.isna().sum()     #checking for null values

In [ ]:
test_data.isna().sum()    #checking for null values

In [ ]:
# Drop duplicates from the training dataset
train_data = train_data.drop_duplicates()

# Drop duplicates from the test dataset
test_data = test_data.drop_duplicates()

In [ ]:
train_data.describe()     #checking statistical info of dataset

In [ ]:
test_data.describe()       #checking statistical info of dataset

In [ ]:
# a heatmap of missing values
plt.figure(figsize=(10, 6))
sns.heatmap(train_data.isnull(), cmap='viridis', cbar=False, yticklabels=False)

# Set plot title
plt.title('Missing Values in Each Column')

# Display the plot
plt.show()

In [ ]:
sns.boxplot(x='Sepssis',y='Age', data=train_data)
plt.show()

In [ ]:
# plot the boxplot to see the outlier of each numerical column
sns.boxplot(data=train_data,orient="v")


In [ ]:
# Pie chart
labels = ['Sepsis', 'Not Sepsis']
#colors
colors = ['#94B3FD', '#F9C5D5']
ax = plt.pie(train_data['Sepssis'].value_counts(), labeldistance=1.15,
labels=labels, colors=colors, autopct='%1.1f%%',
textprops={'fontsize': 16});
plt.title('Proportion of Sepsis', fontsize=25, fontweight = 'bold')
plt.rcParams['figure.figsize'] = [10, 7]
plt.show()

In [ ]:
# Set the style for seaborn
sns.set(style="whitegrid")

# Univariate Analysis: Distribution of Patient Ages
plt.figure(figsize=(12, 6))
sns.histplot(train_data['Age'], bins=20, kde=True, color='skyblue')
plt.title('Distribution of Patient Ages')
plt.xlabel('Age (years)')
plt.ylabel('Frequency')
plt.show()

# Univariate Analysis: Box Plot for Plasma Glucose Levels
plt.figure(figsize=(12, 6))
sns.boxplot(x=train_data['PRG'], color='lightcoral')
plt.title('Box Plot of Plasma Glucose Levels')
plt.xlabel('Plasma Glucose Level (Attribute1)')
plt.show()

# Univariate Analysis: Bar Chart for Insurance Status
plt.figure(figsize=(8, 5))
sns.countplot(x=train_data['Insurance'], palette='pastel')
plt.title('Insurance Status Distribution')
plt.xlabel('Insurance Status')
plt.ylabel('Count')
plt.show()

In [ ]:
# Bivariate Analysis: Scatter Plot of Plasma Glucose Levels and Blood Pressure
plt.figure(figsize=(10, 6))
sns.scatterplot(x=train_data['PRG'], y=train_data['PR'], color='skyblue')
plt.title('Scatter Plot of Plasma Glucose Levels vs. Blood Pressure')
plt.xlabel('Plasma Glucose Level (Attribute1)')
plt.ylabel('Blood Pressure (Attribute3)')
plt.show()

# Bivariate Analysis: Box Plot of Plasma Glucose Levels Grouped by Insurance Status
plt.figure(figsize=(10, 6))
sns.boxplot(x=train_data['Insurance'], y=train_data['PRG'], palette='pastel')
plt.title('Box Plot of Plasma Glucose Levels Grouped by Insurance Status')
plt.xlabel('Insurance Status')
plt.ylabel('Plasma Glucose Level (Attribute1)')
plt.show()

# Bivariate Analysis: Violin Plot of Body Mass Index and Blood Work Result-2
plt.figure(figsize=(10, 6))
sns.violinplot(x=train_data['M11'], y=train_data['SK'], palette='muted')
plt.title('Violin Plot of Body Mass Index vs. Blood Work Result-2')
plt.xlabel('Body Mass Index (Attribute6)')
plt.ylabel('Blood Work Result-2 (Attribute4)')
plt.show()

In [ ]:
# Selecting four numerical columns for the pair plot
selected_numerical_columns = ['PRG', 'PL', 'PR', 'M11']

# Set the style for seaborn
sns.set(style="whitegrid")

# Multivariate Analysis: Pair Plot for Selected Numerical Columns
sns.pairplot(train_data[selected_numerical_columns])
plt.suptitle('Pair Plot for Selected Numerical Columns', y=1.02)
plt.show()

In [ ]:
# Selecting all numerical columns for the correlation heatmap
numerical_columns = train_data.select_dtypes(include=['float64', 'int64'])

# Calculate the correlation matrix
correlation_matrix = numerical_columns.corr()

# Set the style for seaborn
sns.set(style="white")

# Create a heatmap for the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Heatmap for Numerical Columns')
plt.show()

In [ ]:
# Set the style for seaborn
sns.set(style="whitegrid")

# Plot box plots for each numerical column
plt.figure(figsize=(16, 8))
sns.boxplot(data=train_data, orient='h', palette='Set2')
plt.title('Box Plots for Numerical Columns to Identify Outliers')
plt.show()

In [ ]:
# Print column names
print(train_data.columns)

In [ ]:
# Specify the columns of interest
columns_of_interest =  ['PRG', 'PL', 'PR', 'SK', 'TS', 'M11', 'BD2', 'Age', 'Insurance']

# Check if outliers still exist in the columns
outliers_exist = False

for column in columns_of_interest:
    # Calculate the first and third quartiles (Q1 and Q3)
    Q1 = train_data[column].quantile(0.25)
    Q3 = train_data[column].quantile(0.75)

    # Calculate the interquartile range (IQR)
    IQR = Q3 - Q1

    # Define the lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Modify the values in the column to be within the range
    train_data[column] = train_data[column].clip(lower_bound, upper_bound)

    # Check if outliers exist in the column
    if (train_data[column] < lower_bound).any() or (train_data[column] > upper_bound).any():
        outliers_exist = True
        print(f"Outliers still exist in '{column}'.")

if not outliers_exist:
    print("No outliers exist in the specified columns.")

In [ ]:
#checking again for outliers after removing them

# Set the style for seaborn
sns.set(style="whitegrid")

# Plot box plots for each numerical column
plt.figure(figsize=(16, 8))
sns.boxplot(data=train_data, orient='h', palette='Set2')
plt.title('Box Plots for Numerical Columns to Identify Outliers')
plt.show()

### Issues With the Data
* The column names are not very descriptive.

* The target variable 'Sepssis' may have imbalanced classes.

* There are many outliers in some of the numerical columns.

In [ ]:
from scipy.stats import ttest_ind

# Assuming 'Age' is the column representing patient age
age_sepsis_positive = train_data[train_data['Sepssis'] == 'Positive']['Age']
age_sepsis_negative = train_data[train_data['Sepssis'] == 'Negative']['Age']

# Perform t-test
t_stat, p_value = ttest_ind(age_sepsis_positive, age_sepsis_negative, equal_var=False)

# Print results
print(f'T-statistic: {t_stat}')
print(f'P-value: {p_value}')

# Interpret the results
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference in average age.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference in average age.")


## Data Preprocessing

In [ ]:
# Rename the columns
train_data = train_data.rename(columns={
    "PRG": "Plasma_glucose",
    "PL": "Blood_Work_R1",
    "PR": "Blood_Pressure",
    "SK": "Blood_Work_R2",
    "TS": "Blood_Work_R3",
    "M11": "BMI",
    "BD2": "Blood_Work_R4",
    "Age": "Patient_age",
    "Sepssis": "Sepsis"
})

In [ ]:
train_data.head()

In [ ]:
# Check for missing values
print(train_data.isnull().sum())


In [ ]:
# Replace "Positive" with 1 and "Negative" with 0
train_data['Sepsis'] = train_data['Sepsis'].replace({'Positive': 1, 'Negative': 0})

# Print the updated DataFrame
train_data.head(5)

## **Dropping Unnecessary Columns**
* We will remove Blood_work_R2

* We will remove the ID column

* we drop Insurance as well since it isnt a relevant field

In [ ]:
#dropping unnecessary columns

train_data = train_data.drop(['Blood_Work_R2', 'ID', 'Insurance'], axis=1)
train_data.head()

# **Data Preparation**

## **Data Spliting**


In [ ]:
#Splitting data into x_train and x_test

X = train_data.drop('Sepsis', axis=1)
y = train_data['Sepsis']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#checking shape after splitting
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
X_train

In [ ]:
y_train

## **Detecting Data Imbalance**

In [ ]:
# Count and plot the distribution of the original 'Sepsis' classes
sns.countplot(x=y_train, palette='Set2')
plt.title('Original Distribution of Sepsis Classes')
plt.show()

# Check the distribution of values in the 'Sepsis' column
sepsis_distribution = y_train.value_counts()

# Print the distribution
print(sepsis_distribution)

Dataset is imbalanced so we can't use Accuracy Score to choose our model

To solve this issue: we'll Oversample our minority class using RandomOverSampler

In [ ]:

# Apply SMOTE to oversample the minority class
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Count and plot the distribution of the resampled 'Sepsis' classes
plt.figure(figsize=(8, 6))
sns.countplot(x=y_resampled, palette='Set2')
plt.title('Resampled Distribution of Sepsis Classes using SMOTE')
plt.show()

# Check the distribution of values in the 'Sepsis' column
sepsis_distribution = y_resampled.value_counts()

# Print the distribution
print(sepsis_distribution)

## **Feature Scaling**

In [ ]:
# Create an instance of StandardScaler and set output to be a DataFrame
scaler = StandardScaler().fit(X_train).set_output (transform="pandas")

# Scale the training data
X_train_df = scaler.transform(X_train)

# Scale the test data using the same scaler
X_test_df = scaler.transform(X_test)

## **Save Preprocessed Data**

In [ ]:
# # Save the preprocessed data to a new CSV file
# X_train_df.to_csv('preprocessed_data.csv', index=False)


# MODELLING

Here is the section to build, train, evaluate and compare the models to each other.
* Logistic Regression
* RandomForest Classifier
* XGBoost Classifier
* K Nearest Neighbors
* Support Vector Machines
* DecisionTreeClassifier
* Gradient Boosting Classifier Model

In [ ]:
# Initialize models
models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'XGBoost': XGBClassifier(),
    'K Nearest Neighbors': KNeighborsClassifier(),
    'Support Vector Machines': SVC(),
    'Decision Tree': DecisionTreeClassifier(),
    'Gradient Boosting': GradientBoostingClassifier()
}

In [ ]:
models

# EVALUATION

In [ ]:
# Initialize DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

# Train and evaluate each model
for name, model in models.items():
    print(f"\nEvaluating {name}...")

    # Train the model
    model.fit(X_train, y_train)

    # Predictions
    y_pred = model.predict(X_test)

    # Evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='binary')  # Add 'average' parameter
    recall = recall_score(y_test, y_pred, average='binary')  # Add 'average' parameter
    f1 = f1_score(y_test, y_pred, average='binary')  # Add 'average' parameter
    roc_auc = roc_auc_score(y_test, y_pred)

    # Add results to DataFrame
    results_df = results_df.append({'Model': name, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1 Score': f1, 'ROC AUC': roc_auc}, ignore_index=True)


In [ ]:
# Display the results DataFrame
print("\nResults Summary:")
results_df

In [ ]:
# Calculate confusion matrix
matrix = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

## Models Comparison

In [ ]:
# Set the style for seaborn
sns.set(style="darkgrid")

# Plot the accuracy of each model using seaborn
plt.figure(figsize=(12, 8))
sns.barplot(x='Model', y='Accuracy', data=results_df, color='green')
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('Accuracy of Each Model')
plt.ylim(0, 1)  # Set y-axis limit between 0 and 1 for accuracy
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better visibility
plt.tight_layout()

# Display the plot
plt.show()

## **k-Fold Cross Validation**

In [ ]:
def kfold_cross_validation(model, X, y, num_folds=5):
    """
    Perform k-fold cross-validation for a given model.

    Parameters:
    - model: Machine learning model.
    - X: Features.
    - y: Target variable.
    - num_folds: Number of folds for cross-validation.

    Returns:
    - results_df: DataFrame containing cross-validation results.
    """
    # Initialize StratifiedKFold
    kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

    # Initialize DataFrame to store results
    results_df = pd.DataFrame(columns=['Fold', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC'])

    # Iterate over folds
    for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Fit the model on the training data
        model.fit(X_train, y_train)

        # Make predictions on the test data
        y_pred = model.predict(X_test)

        # Evaluate the model
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred)

        # Append results to the DataFrame
        results_df = results_df.append({
            'Fold': fold + 1,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1,
            'ROC AUC': roc_auc
        }, ignore_index=True)

    return results_df

In [ ]:
# def kfold_cross_validation_for_models(models, X, y, num_folds=5):
#     """
#     Perform k-fold cross-validation for each model.

#     Parameters:
#     - models: Dictionary of models to evaluate.
#     - X: Features.
#     - y: Target variable.
#     - num_folds: Number of folds for cross-validation.

#     Returns:
#     - results_df: DataFrame containing cross-validation results for each model.
#     """
#     # Initialize DataFrame to store results
#     results_df = pd.DataFrame(columns=['Model', 'Fold', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC'])

#     # Iterate over models
#     for model_name, model in models.items():
#         print(f"\nEvaluating {model_name}...")

#         # Perform k-fold cross-validation for the current model
#         cv_results = kfold_cross_validation(model, X, y, num_folds=num_folds)

#         # Add model name to the results
#         cv_results['Model'] = model_name

#         # Append results to the main DataFrame
#         results_df = results_df.append(cv_results, ignore_index=True)

#     return results_df


In [ ]:
# Example usage
# Replace RandomForestClassifier() and XGBClassifier() with your desired models
models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'XGBoost': XGBClassifier(),
    'K Nearest Neighbors': KNeighborsClassifier(),
    'Support Vector Machines': SVC(),
    'Decision Tree': DecisionTreeClassifier(),
    'Gradient Boosting': GradientBoostingClassifier()
}

cv_results_all_models = kfold_cross_validation_for_models(models, X, y, num_folds=5)

# Display the cross-validation results DataFrame for all models
print("\nCross-Validation Results for All Models:")
cv_results_all_models

In [ ]:
# Filter the DataFrame to include only the accuracy results
accuracy_results_all_models = cv_results_all_models[['Model', 'Accuracy']]

# Plot the bar chart
plt.figure(figsize=(12, 8))
sns.barplot(x='Model', y='Accuracy', data=accuracy_results_all_models, palette='viridis')
plt.title('Model Comparison - 5-Fold Cross-Validation Accuracy')
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.ylim(0, 1)  # Set the y-axis limit to the range of accuracy (0 to 1)
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.show()

In [ ]:
# Filter the DataFrame to include only the accuracy results for the 5th fold
accuracy_results_5th_fold = cv_results_all_models[cv_results_all_models['Fold'] == 5][['Model', 'Accuracy']]

# Print the accuracy for the 5th fold for each model
print("\nAccuracy for the 5th Fold:")
accuracy_results_5th_fold

In [ ]:
# Calculate the mean accuracy for each model
mean_accuracy_by_model = cv_results_all_models.groupby('Model')['Accuracy'].mean()

# Select the top two models with the highest mean accuracy
top_two_models = mean_accuracy_by_model.nlargest(2)

# Print the two best models and their mean accuracies
print("\nTwo Best Models based on k-fold Cross-Validation:\n")
for model, mean_accuracy in top_two_models.items():
    print(f"Model: {model}, Mean Accuracy: {mean_accuracy}")

# **Hyperparameters Tuning**

In [ ]:
# # Create a pipeline with a model (RandomForestClassifier)
# model = RandomForestClassifier()

# # Define the hyperparameter grid to search
# param_grid = {
#     'n_estimators': [50, 100, 150],
#     'max_depth': [None, 10, 20],
#     'min_samples_split': [2, 6, 10],
#     'min_samples_leaf': [1, 2, 4]
# }

# # Create the GridSearchCV object
# grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# # Fit the grid search to the data
# grid_search.fit(X_train, y_train)

# # Print the best hyperparameters and corresponding accuracy
# print("Best Hyperparameters:")
# print(grid_search.best_params_)
# print("Best Accuracy on Training Data:", grid_search.best_score_)

# # Evaluate the best model on the test set
# best_model = grid_search.best_estimator_
# test_accuracy = best_model.score(X_test, y_test)
# print("Accuracy on Test Data:", test_accuracy)

In [ ]:
# Create hyperparameter grids for each model
param_grid_rf = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': ['auto', 'sqrt', 'log2']
}

param_grid_lr = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

# Create the GridSearchCV objects for each model
grid_search_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_lr = GridSearchCV(LogisticRegression(), param_grid_lr, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the grid searches to the data
grid_search_rf.fit(X_train, y_train)
grid_search_lr.fit(X_train, y_train)

# Print the best hyperparameters and corresponding accuracy for each model
print("Best Hyperparameters for RandomForestClassifier:")
print(grid_search_rf.best_params_)
print("Best Accuracy on Training Data:", grid_search_rf.best_score_)

print("\nBest Hyperparameters for LogisticRegression:")
print(grid_search_lr.best_params_)
print("Best Accuracy on Training Data:", grid_search_lr.best_score_)

# Evaluate the best models on the test set
best_model_rf = grid_search_rf.best_estimator_
test_accuracy_rf = best_model_rf.score(X_test, y_test)
print("\nAccuracy on Test Data (RandomForestClassifier):", test_accuracy_rf)

best_model_lr = grid_search_lr.best_estimator_
test_accuracy_lr = best_model_lr.score(X_test, y_test)
print("Accuracy on Test Data (LogisticRegression):", test_accuracy_lr)

## **Exporting Model and Preprocessor**

In [ ]:
# Save the trained model
joblib.dump(best_model_rf, 'best_model_rf.joblib')

# Save the preprocessor
joblib.dump(scaler, 'scaler.joblib')

In [ ]:
#get requirements
!pip freeze > requirements.txt
